# Lesson I

## Using Context Managers

In this lesson, I'll introduce the concept of context managers and show you how to use these special kinds of functions.

### What is a context manager?

A context manager is a type of function that *sets up a context* for your code to run in, *runs your code*, and *then removes the context*. That's not a very helpful definition though, so let me explain with an analogy.

Imagine that you are throwing a fancy party, and have hired some caterers to provide refreshments for your guests.

* Before the party starts, the caterers set up tables with food and drinks.
* Then you and your friends dance, eat, and have a good time.
* Finally, the caterers clean up the tables and leave.

In this analogy, the caterers are like a context manager...

| **Context Manager** | **Caterers** |
| -------------------- | ------------ |
| * Set up a context * | * Set up tables * |
| * Run your code * | * Dance, eat, and have a good time * |
| * Remove the context * | * Clean up tables * |

#### A real_world_example

You may have used code like this before:

```python
    with open('file.txt') as my_file:
        text = my_file.read()
        length = len(text)

    print('The file is {} characters long.'.format(length))    
```

In this example, we read the text of the file, store the contents of the file in the variable ``"text"``, and store the length of the contents in the variable ``"length"``. 
When the code inside the indented block is done, the ``"open()"`` function makes sure that the file is closed before continuing on in the script. 
The *print statement* is outside of the context, so by the time it runs the file is closed.

***``open()``does three things:***
* Sets up a context by opening a file.
* Lets you run any code you want on that file
* Removes the context by closing the file.

### Using a context manager

Any time you use a context manager, it will look like this.

```python
    with <context-manager>(<args>) as <variable-name>:
        # Run your code here
        # This code is running "inside the context"

    # This code runs after the context is removed"    
```

* The keyword ``"with"`` lets Python know that you are trying to enter a context.
* Then you call a function. You can call any function that is built to work as a context manager. 
* A context manager can take arguments like any normal function.
* You end the "with" statement with a colon as if you were writing a for loop or an if statement.


Statements in Python that have an indented block after them, like for loops, if/else statements, function definitions, etc. are called **"compound statements"**. 

The ``"with"`` statement is another type of compound statement. Any code that you want to run inside the context that the context manager created needs to be indented.

When the indented block is done, the context manager gets a chance to clean up anything that it needs to, like when the ``"open()"`` context manager closed the file.

Some context managers want to return a value that you can use inside the context. By adding ``"as"`` and a variable name at the end of the *"with"* statement, you can assign the returned value to the variable name. 

```python
    with open('file.txt') as my_file:
        text = my_file.read()
        length = len(text)

    print('The file is {} characters long.'.format(length)))    
```

We used this ability when calling the ``"open()"`` context manager, which returns a file that we can read from or write to. 
By adding ``"as my_file"`` to the *"with"* statement, we assigned the file to the variable ``"my_file"``.

# Lesson II

## Writing Context Managers

Now that you know how to use context managers, I want to show you how to write a context manager for other people to use.

### Two ways to define a context manager

* Class based
    - __enter__(), and __exit__() methods

* **Function based**

#### How to create a context manager

There are five parts to creating a context manager:

1. Define a function
2. (optional) Add any set up code your context needs.
3. Use the ``"yield"`` keyword.
4. (optional) Add any teardown code your context needs.

```python
    def my_context():
        # Add any set up code your context needs.
        yield
        # Add any teardown code your context needs.
```

***Finally, you must decorate the function with the ``"contextmanager"`` decorator from the ``"contextlib"`` module.***

```python
    @contextlib.contextmanager
    def my_context():
        # Add any set up code your context needs.
        yield
        # Add any teardown code your context needs.
```

### The "Yield" Keyword

The *"yield"* keyword may also be new to you. When you write this word, it means that you are going to return a value, but you expect to finish the rest of the function at some point in the future.

The value that your context manager yields can be assigned to a variable in the *"with"* statement by adding ``"as <variable name>"``

In [3]:
import contextlib
@contextlib.contextmanager
def my_context():
    print('Hello')
    yield 42
    print('Goodbye')

In [4]:
with my_context() as foo:
    print('foo is {}'.format(foo))

Hello
foo is 42
Goodbye


Here, we've assigned the *value 42* that ``my_context()`` yields to the variable ``"foo"``. 
By running this code, you can see that after the context block is done executing, the rest of the ``my_context()`` function gets run, *printing "goodbye"*. 

You may recognize the ``"yield"`` keyword as a thing that gets used when creating **generators**. In fact, a context manager function is technically a generator that yields a single value.

### Setup and Teardown

The ability for a function to yield control and know that it will get to finish running later is what makes context managers so useful. This context manager is an example of code that accesses a database. 

```python
    @contextlib.contextmanager
    def database(url):
        # Set up datase connection
        db = postgres.connect(url)

        yield db

        # Tear down database connection
        db.disconnect()
```

Like most context managers, it has some setup code that runs before the function yields. This context manager uses that setup code to connect to the database.

Most context managers also have some teardown or cleanup code when they get control back after yielding. This one uses the teardown section to disconnect from the database.

```python
    url = 'http://datacamp./data'
    with databas(url) as my_db:
        course_list = my_db.execute(
            'SELECT * FROM courses'
        )
```

This setup/teardown behavior allows a context manager to hide things like connecting and disconnecting from a database so that a programmer using the context manager can just perform operations on the database without worrying about the underlying details.

### Yielding a value or None

The ``database()`` context manager that we've been looking at yields a specific value - *the database connection *- that can be used in the context block. 
Some context managers don't yield an explicit value.

```python
    @contextlib.contextmanager
    def in_dir(path):
        # Save current working directory
        old_dir = os.getcwd()

        # Switch to new working directory
        os.chdir(path)

        yield

        # Change back to previous working directory
        os.chdir(old_dir)
```

``in_dir()`` is a context manager that changes the current working directory to a specific path and then changes it back after the context block is done. It does not need to return anything with its *"yield"* statement.